<a href="https://colab.research.google.com/github/Kavyabikkasani/AI_Internship_oct24/blob/main/Amazon_Return_Policy_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y protobuf
!pip install protobuf==3.20.3
!pip install -q groq langchain-groq litellm PyPDF2 chromadb text-generation langchain sentence-transformers langchain-community rank_bm25 gradio


Found existing installation: protobuf 4.25.5
Uninstalling protobuf-4.25.5:
  Successfully uninstalled protobuf-4.25.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 13.7 MB/s eta 0:0

In [2]:
print("Testing imports...")
import os
import gradio as gr
from typing import Dict, List, Optional
from google.colab import files, userdata

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_groq import ChatGroq
print("All imports successful!")

Testing imports...
All imports successful!


In [3]:
class AmazonReturnPolicyAssistant:
    def __init__(self, api_key: str):
        print("Initializing assistant...")
        os.environ["GROQ_API_KEY"] = api_key
        print("API key set")

        print("Setting up LLM...")
        self.llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
        print("LLM setup complete")

        print("Setting up embeddings...")
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-l6-v2",
            model_kwargs={'device': 'cpu'},
            encode_kwargs={'normalize_embeddings': False}
        )
        print("Embeddings setup complete")
        print("Initialization complete!")

    def process_document(self, file_path: str):
        print(f"Processing document: {file_path}")
        loader = TextLoader(file_path)
        data = loader.load()
        print("Document loaded")

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=100,
            chunk_overlap=50
        )
        docs = text_splitter.split_documents(data)
        print("Document split into chunks")

        vectordb = Chroma.from_documents(docs, self.embeddings)
        vector_retriever = vectordb.as_retriever(
            search_type="mmr",
            search_kwargs={"k": 1}
        )
        print("Vector retriever created")

        bm25_retriever = BM25Retriever.from_documents(docs)
        bm25_retriever.k = 2
        print("BM25 retriever created")

        ensemble_retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, vector_retriever],
            weights=[0.2, 0.8]
        )
        print("Ensemble retriever created")

        prompt = PromptTemplate(
            template="""You are an Amazon Return Policy Assistant, expert in providing clear and concise information.
            Context: {context}
            Question: {question}
            Please provide a clear and helpful response.""",
            input_variables=["context", "question"]
        )
        print("Prompt template created")

        qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=ensemble_retriever,
            chain_type_kwargs={"prompt": prompt}
        )
        print("QA chain created")
        return qa_chain

In [4]:
print("Testing file upload...")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
print(f"File uploaded: {file_name}")

print("Getting API key...")
groq_api_key = userdata.get('groq_api')
if not groq_api_key:
    raise ValueError("GROQ API key not found in Colab secrets")
print("API key retrieved")

Testing file upload...


Saving return policy.txt to return policy (1).txt
File uploaded: return policy (1).txt
Getting API key...
API key retrieved


In [5]:
print("Creating assistant...")
assistant = AmazonReturnPolicyAssistant(groq_api_key)
qa_chain = assistant.process_document(file_name)
print("Assistant ready!")

Creating assistant...
Initializing assistant...
API key set
Setting up LLM...
LLM setup complete
Setting up embeddings...


<ipython-input-3-a8ad1e4c1719>:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings setup complete
Initialization complete!
Processing document: return policy (1).txt
Document loaded
Document split into chunks
Vector retriever created
BM25 retriever created
Ensemble retriever created
Prompt template created
QA chain created
Assistant ready!


In [6]:
def create_interface(qa_chain):
    def query_handler(query: str) -> str:
        try:
            return qa_chain.run({"query": query})
        except Exception as e:
            return f"Error processing query: {str(e)}"

    interface = gr.Interface(
        fn=query_handler,
        inputs=gr.Textbox(
            lines=4,
            placeholder="Ask about Amazon's return policies...",
            label="Your Question"
        ),
        outputs=gr.Textbox(label="Answer"),
        title="Amazon Return Policy Assistant",
        examples=[["What is the general return policy?"]]
    )
    return interface

print("Launching interface...")
interface = create_interface(qa_chain)
interface.launch()

Launching interface...
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c08d57bb1d30a3ef17.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
